<a href="https://colab.research.google.com/github/drawcodeboy/first_ML_DL/blob/main/firstMLDL_05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tree Ensemble 
>* Random Forest
>* Extra Tree
>* Gradient Boosting
>* Histogram-based Gradient Boosting

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('http://bit.ly/wine_csv_data')
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42
)

# Random Forest
>* Bootstrap Sample
>* 분류 모델은 노드를 분할 할 때, 전체 특성 개수의 제곱근 만큼 랜덤하게 가져오고, 회귀는 전체 특성을 사용한다.
>* 샘플(Bootstrap)을 랜덤, 특성 또한 랜덤하게 가져오기 때문에 훈련 세트에 Overfitting 되는 것을 막아준다.
>* 검증 세트와 테스트 세트에서도 안정적인 성능을 낼 수 있다.
>* 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이하다.
>* OOB(Out of bag) Sample

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
# 훈련 세트, 검증 세트 점수 비교
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # test_score = 교차 검증 점수

0.9973541965122431 0.8905151032797809


In [7]:
# 특성 중요도 파악

# 당도의 중요도가 감소하고 다른 특성의 중요도들이 증가한 것을 파악할 수 있다.
# (단일 트리 모델에 비해서)
# Random Forest가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다.
# 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [8]:
# OOB(Out of bag) Sample, Bootstrap Sample에 이용되지 않고 남은 샘플이다. 
# 이 샘플에 대한 평가를 검증 세트의 역할로 가져올 수 있다.

rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state =42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# Extra Tree
>* Random Forest와 달리 Bootstrap Sample을 사용하지 않는다.
>* 즉, 결정 트리를 만들 때 전체 훈련 세트를 사용
>* 노드를 분할할 때, 가장 좋은 분할이 아닌 무작위로 분할
>* 성능은 낮아지지만 Overfitting을 막고 검증 세트에 대한 점수를 높인다.
>* 보통 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 많은 결정 트리를 훈련해야 한다.
>* 랜덤하게 노드를 분할하기 때문에 계산속도는 랜덤 포레스트보다 빠르다.

In [9]:
# Extra Tree

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [10]:
et.fit(train_input, train_target)
print(et.feature_importances_)
# Decision Tree보다 당도에 대한 의존성이 낮아짐

[0.20183568 0.52242907 0.27573525]


# Gradient Boosting
>* 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블한다.
>* 깊이가 얕기 때문에 Overfitting에 강하고, 높은 일반화 성능을 기대할 수 있다.
>* 경사하강법 사용 (Classification = Logistic Loss Function, Regression = Mean Square Error Function)
>* 학습률을 증가시키고, 트리의 개수를 증가시켜도 되려 성능이 향상되고 Overfitting은 거의 일어나지 않는다.
>* subsample 매개변수를 통해 훈련 세트의 비율을 정할 수 있는데 이를 조정함으로써 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷하게 할 수 있다.

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [13]:
# n_estimators(Decision Tree 개수)를 100 -> 500
# learning_rate를 0.1 -> 0.2
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [15]:
# 일부 feature(당도)에 더 집중한다.
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# Histogram-Based Gradient Boosting
>* 입력 특성을 256개의 구간으로 나눈다.
>* 즉, 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
>* 트리 개수 지정 매개변수 = max_iter
>* Histogram-Based Gradient Boosting을 제공하는 라이브러리: XGBoost, LightGBM

### + permutation_importance()
>* 특성 중요도를 계산하기 위한 함수
>* 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산
>* 사이킷런에서 제공하는 모든 Estimator 모델에 사용가능하다.
>* 반환하는 객체: importances, importances_mean, importances_std

In [16]:
# Histogram-Based Gradient Boosting

from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [17]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, 
                                n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [18]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [21]:
hgb.score(test_input, test_target)

# 87%로 RandomSearch에서는 86%이었지만
# 앙상블 모델이 단일 트리 모델에서보다 성능이 좋다는 것으로 판단할 수 있다.

0.8723076923076923

In [22]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [23]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
